In [6]:
# Importaciones de la biblioteca estándar
import sys
import time
import pickle
import sqlite3

# Importaciones de terceros
import numpy as np
import pandas as pd
import torch
import torchaudio
from scipy.signal import butter, sosfilt
from tqdm.notebook import tqdm

# Importaciones de módulos específicos del proyecto
from maad import sound, features
import indices as indices

# Añadir el directorio utils al PATH para importar otros módulos si es necesario
sys.path.append("../utils/")

# Configuración de la conexión a la base de datos
conn = sqlite3.connect('../results/results.db')
cursor = conn.cursor()


In [2]:
def butter_bandstop_filter(audio_signal, frequencies_start, frequencies_end, fs, order=4):
    """
    Apply a series of higher-order Butterworth bandstop filters to an audio signal.

    Parameters:
    - audio_signal: Numpy array containing the audio signal to be filtered.
    - frequencies_start: List of start frequencies for the bandstop filters.
    - frequencies_end: List of end frequencies for the bandstop filters.
    - fs: Sampling frequency of the audio signal.
    - order: Order of the Butterworth filters.

    Returns:
    - audio_signal: The filtered audio signal.
    """
    # Ensure the frequency lists are of the same length
    if len(frequencies_start) != len(frequencies_end):
        raise ValueError("Start and end frequency lists must have the same length.")

    # Design and apply each bandstop filter
    for start_freq, end_freq in zip(frequencies_start, frequencies_end):
        # Calculate the Nyquist frequency
        nyq = fs / 2.0
        # Calculate low and high cutoff frequencies for the bandstop filter
        low = start_freq / nyq
        high = end_freq / nyq
        # Design the Butterworth bandstop filter
        sos = butter(order, [low, high], btype='bandstop', fs=fs, output='sos')
        # Apply the filter to the audio signal
        audio_signal = sosfilt(sos, audio_signal)

    return audio_signal

def get_audio_indices(file_name, apply_filter=False, start_freqs=None, end_freqs=None):
    """
    Load an audio file, optionally apply a bandstop filter, and calculate various acoustic indices.

    Parameters:
    - file_name: string, path to the audio file.
    - apply_filter: boolean, indicates if a bandstop filter should be applied.
    - start_freqs: list of floats, start frequencies for the bandstop filters, used if apply_filter is True.
    - end_freqs: list of floats, end frequencies for the bandstop filters, used if apply_filter is True.

    Returns:
    - indices: list, containing the file name and computed acoustic indices or np.nan in case of an error.
    """

        # Load the audio file and compute the spectrogram
    s, fs = torchaudio.load(str(file_name))
    if s.size()[0] != 1:
        s = torch.unsqueeze(s[0, :], 0)

        # If filtering is applied, filter the signal before computing the spectrogram
    if apply_filter and start_freqs is not None and end_freqs is not None:
        s_filtered = butter_bandstop_filter(s.numpy()[0, :], start_freqs, end_freqs, fs)
        s = torch.tensor(s_filtered.copy()[None, :])  # Add a new axis to make it 2D again
        rms = np.sqrt(np.mean(np.square(s.numpy()[0, :])))
        s = s / rms # Normalize the signal

        # Compute the spectrogram
    Sxx, tn, fn, ext = sound.spectrogram(s[0, :], fs, mode='amplitude')
    Sxx_power, _, _, _ = sound.spectrogram(s[0, :], fs)

        # Calculate acoustic indices
    ACIft_ = indices.ACIft(Sxx)
    ADI = features.acoustic_diversity_index(Sxx, fn, fmax=int(fs/2))
    BETA = features.bioacoustics_index(Sxx, fn, flim=(2000, 8000))
    M = features.temporal_median(s[0, :], mode='hilbert')
    NP = features.number_of_peaks(Sxx_power, fn, slopes=6, min_freq_dist=100, display=False)
    Hf, _ = features.frequency_entropy(Sxx_power)
    Ht = features.temporal_entropy(s.numpy()[0, :], mode='hilbert')
    H = Ht * Hf
    AEI = features.acoustic_eveness_index(Sxx, fn, fmax=int(fs/2))
    NDSI, _, _, _ = features.soundscape_index(Sxx_power, fn, flim_bioPh=(2000, 8000), flim_antroPh=(0, 2000))

        # Compile all indices into a list
    acoustic_indices = [ACIft_, ADI, BETA, M, NP, H, AEI, NDSI]

        # Include file name information in the indices list
    file_info = [str(file_name).split('/')[-1], str(file_name).split('/')[-2]]
    return file_info + acoustic_indices




In [5]:
unique_folders = ['SM04', 'G0059','G0096','G0001']

for x, xfolder in tqdm(enumerate(unique_folders), total=len(unique_folders)):
    for y, yfolder in enumerate(unique_folders[x+1:]):
        max_attempts = 5
        attempt_delay = 2  # Tiempo de espera inicial en segundos

        for attempt in range(max_attempts):
            try:
                cfr_query = f"SELECT * FROM comparable_frequency_range WHERE folder_x = ? AND folder_y = ?"
                cfr = pd.read_sql_query(cfr_query, conn, params=(xfolder, yfolder))
                if cfr.empty:
                    cfr = pd.read_sql_query(cfr_query, conn, params=(yfolder, xfolder))
                    xfolder, yfolder = yfolder, xfolder
                
                # Deserializar las frecuencias
                cfr['start_freq'] = cfr['start_freq'].apply(pickle.loads)
                cfr['end_freq'] = cfr['end_freq'].apply(pickle.loads)
                
                # Get the indices for each file
                xaudios = pd.read_sql_query("SELECT path FROM audios WHERE folder = ?", conn, params=(xfolder,))
                yaudios = pd.read_sql_query("SELECT path FROM audios WHERE folder = ?", conn, params=(yfolder,))

                for xpath, ypath in zip(xaudios.path, yaudios.path):
                    # Suponemos que la función get_audio_indices está definida en otro lugar del código
                    rxind = get_audio_indices(xpath)
                    pxind = get_audio_indices(xpath, apply_filter=True, start_freqs=cfr.start_freq.iloc[0], end_freqs=cfr.end_freq.iloc[0])
                    ryind = get_audio_indices(ypath)
                    pyind = get_audio_indices(ypath, apply_filter=True, start_freqs=cfr.start_freq.iloc[0], end_freqs=cfr.end_freq.iloc[0])

                    insert_query = "INSERT INTO acoustic_indices (folder_x, folder_y, rx_indices, px_indices, ry_indices, py_indices) VALUES (?, ?, ?, ?, ?, ?)"
                    cursor.execute(insert_query, (xfolder, yfolder, pickle.dumps(rxind), pickle.dumps(pxind), pickle.dumps(ryind), pickle.dumps(pyind)))
                    conn.commit()
    
                # Si llegamos aquí sin excepciones, rompemos el bucle de intentos
                break

            except sqlite3.OperationalError as e:
                # Si es el último intento, levanta la excepción
                if attempt == max_attempts - 1:
                    raise
                print(f"OperationalError encountered on attempt {attempt+1}: {e}")
                print(f"Waiting for {attempt_delay} seconds before retrying...")
                time.sleep(attempt_delay)
                attempt_delay *= 2  # Incrementar el tiempo de espera para el próximo intento

conn.close()

            



  0%|          | 0/4 [00:00<?, ?it/s]